In [1]:
import torch
from torch.utils.data import Dataset

import torchaudio
import torchaudio.transforms

import sys, os

from pprint import pprint

from tqdm.autonotebook import tqdm

import json

import numpy as np

import matplotlib.pylab as plt
import seaborn as sns

import librosa
import librosa.display

import pandas as pd

from pathlib import Path

import gc

MANUAL_SEED = 69

import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim

from datetime import date
from datetime import datetime

import os.path
from os import path
  
import json

import time

import copy

from matplotlib import pyplot as plt
plt.rcParams['figure.dpi'] = 150
plt.rcParams['savefig.dpi'] = 150

from sklearn.decomposition import PCA



In [2]:
def make_dir_if_absent(dir_path):
  
  # print("making dir: ", dir_path)
  
  if not os.path.exists(dir_path):
    os.makedirs(dir_path)
    

In [3]:
class FMADataset(Dataset):

  def __init__(
    self, path_waveform, path_mel_spectrogram
  ):
    self.path_waveform = path_waveform
    self.path_mel_spectrogram = path_mel_spectrogram
    
    self.data_paths_waveform = self._load_audio_list(self.path_waveform)
    self.data_paths_mel_spectrogram = self._load_audio_list(self.path_mel_spectrogram)

  def __len__(self):
    return len(self.data_paths)

  def __getitem__(self, idx):
    
    data_waveform = torch.load(self.data_paths_waveform[idx])
    data_mel_spectrogram = torch.load(self.data_paths_mel_spectrogram[idx])

    return data_waveform, data_mel_spectrogram
    
  def _load_audio_list(self, waveform_or_mel_spec_path):
    
    audio_path_list = []
    
    for path, subdirs, files in tqdm(
      os.walk(waveform_or_mel_spec_path), colour="magenta"
    ):
      for name in files:
          
        file_audio_path = os.path.join(path, name)
        
        audio_path_list.append(file_audio_path)
        
    return audio_path_list
        
        

In [4]:
DATASET_SIZE = "s"
DATASET_FOLDER_WAVEFORM = f"../data/waveform"
DATASET_FOLDER_MEL_SPECTROGRAM = f"../data/mel_spectrogram"

DATASET_NUM_SAMPLES_PER_SECOND = 22050
DATASET_NUM_CHANNELS = 1

DATASET_NAME = f"fma_{DATASET_SIZE}_resampled_{DATASET_NUM_SAMPLES_PER_SECOND}_rechanneled_{DATASET_NUM_CHANNELS}"

DATASET_NAME_WAVEFORM = DATASET_NAME

MEL_SPECTROGRAM_N_FFT = 1024
MEL_SPECTROGRAM_WIN_LENGTH = None
MEL_SPECTROGRAM_HOP_LENGTH = 128
MEL_SPECTROGRAM_N_MELS =  128
DATASET_NAME_MEL_SPECTROGRAM = f"fma_{DATASET_SIZE}_resampled_{DATASET_NUM_SAMPLES_PER_SECOND}_rechanneled_{DATASET_NUM_CHANNELS}_n_fft_{MEL_SPECTROGRAM_N_FFT}_win_length_{MEL_SPECTROGRAM_WIN_LENGTH}_hop_length_{MEL_SPECTROGRAM_HOP_LENGTH}_n_mels_{MEL_SPECTROGRAM_N_MELS}"

dataset_path_waveform = f"{DATASET_FOLDER_WAVEFORM}/{DATASET_NAME_WAVEFORM}"
dataset_path_mel_spectrogram = f"{DATASET_FOLDER_MEL_SPECTROGRAM}/{DATASET_NAME_MEL_SPECTROGRAM}"

TRAINING_LOGS_FOLDER = "./logs"



In [5]:
fma_dataset = FMADataset(
  path_waveform=dataset_path_waveform, 
  path_mel_spectrogram=dataset_path_mel_spectrogram
)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [6]:
list_waveforms = []
list_mel_spectrograms = []

In [7]:
for waveform, mel_spectrogram in tqdm(fma_dataset, colour="#32a852"):
  # list_waveforms.append(waveform)
  list_mel_spectrograms.append(mel_spectrogram)

0it [00:00, ?it/s]

In [8]:
tensor_waveforms = torch.cat(list_waveforms)
tensor_mel_spectrograms = torch.cat(list_mel_spectrograms)

print(tensor_waveforms.shape)
print(tensor_mel_spectrograms.shape)

torch.Size([3000, 128, 5157])


In [9]:
waveform_std, waveform_mean = torch.std_mean(tensor_waveforms)
mel_spectrogram_std, mel_spectrogram_mean = torch.std_mean(
  tensor_mel_spectrograms
)

In [10]:
to_export = {
  "dataset_name": DATASET_NAME,
  "waveform_mean": waveform_mean.item(),
  "waveform_std": waveform_std.item(),
  "mel_spectrogram_mean": mel_spectrogram_mean.item(),
  "mel_spectrogram_std": mel_spectrogram_std.item()
}

In [11]:
SUMMARY_STATISTICS_FOLDER = "../data/summary_statistics"

SUMMARY_STATISTICS_NAME = DATASET_NAME

summary_statistics_path = f"{SUMMARY_STATISTICS_FOLDER}/{SUMMARY_STATISTICS_NAME}/{SUMMARY_STATISTICS_NAME}_summary_statistics.json"



In [12]:
make_dir_if_absent(
  dir_path=f"{SUMMARY_STATISTICS_FOLDER}/{SUMMARY_STATISTICS_NAME}"
)

In [13]:
with open(summary_statistics_path, 'w') as fp:
  json.dump(to_export, fp)

In [14]:
to_export

{'dataset_name': 'fma_s_resampled_22050_rechanneled_1',
 'waveform_mean': -0.00016133548342622817,
 'waveform_std': 0.30404531955718994,
 'mel_spectrogram_mean': 6.5207295417785645,
 'mel_spectrogram_std': 59.876766204833984}